In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply()
c = twint.Config()

def scrape_user():
    print ("Fetching Tweets")
    c = twint.Config()
    c.Username = input('Enter the id name: ') 
    c.Language = 'en'
    # choose beginning time (narrow results)
    # c.Since = input('Date (format: "%Y-%m-%d %H:%M:%S"): ')
    c.Pandas = True
    c.Limit = 100
    twint.run.Search(c)
    tweets = twint.storage.panda.Tweets_df
    return tweets


In [ ]:
tt = scrape_user()
tlist = []

Fetching Tweets
Enter the id name: Cristiano
1418257821041414145 2021-07-22 17:13:09 +0000 <Cristiano> My beautiful queen ❤️  https://t.co/Jmbld6ySq7
1416076947680026633 2021-07-16 16:47:08 +0000 <Cristiano> A good weekend to everyone 😎🙏🏽  https://t.co/BRklRcvbxI
1414933377434767361 2021-07-13 13:03:00 +0000 <Cristiano> My love 💙  https://t.co/PH7gPrWYxT
1414194896219648000 2021-07-11 12:08:32 +0000 <Cristiano> Se você tiver o “ter” sem o “ser”, não pertence a você. 😉 #LR  https://t.co/hD7jvmmUew
1413187283512176642 2021-07-08 17:24:38 +0000 <Cristiano> Peace of mind 🌊  https://t.co/LuCcZ8DlaC
1412804774886612994 2021-07-07 16:04:41 +0000 <Cristiano> What a magical date - 07.07 to introduce you to my new hotel @PestanaCR7 Times Square, in New York 🤩!  Book now and discover the coolest new hotel, in the best location in town!  #PestanaCR7TimesSquare #PestanaCR7  https://t.co/iD2JL8wPVA
1412517448222400517 2021-07-06 21:02:57 +0000 <Cristiano> My baby boy ❤️🙏🏽  https://t.co/GboZaWgoKf
14

In [ ]:
tt['tweet']

0        My beautiful queen ❤️  https://t.co/Jmbld6ySq7
1     A good weekend to everyone 😎🙏🏽  https://t.co/B...
2                    My love 💙  https://t.co/PH7gPrWYxT
3     Se você tiver o “ter” sem o “ser”, não pertenc...
4              Peace of mind 🌊  https://t.co/LuCcZ8DlaC
                            ...                        
95    Gracias por llegar a nuestras vidas y llenarlo...
96    Feliz por atingir mais uma marca histórica com...
97    Feliz por estar de volta a casa! 🇵🇹💪🏽 #todospo...
98    Head up and believe in the work we’re doing! 💪...
99    Excited about this collaboration with @clear ....
Name: tweet, Length: 100, dtype: object

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/best_model')

# view model architecture to confirm we have save and loaded correctly
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (Custom>TFBertMainLayer)   {'last_hidden_state' 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [ ]:
# def sentiment_score(tweet):
#     tokens = tokenizer.encode(tweet, return_tensors='pt')
#     result = model(tokens)
#     return int(torch.argmax(result.logits))+1

In [ ]:
# data = [input("Input your mail text")]

In [ ]:
# infl = load_model.predict(data)[0]

In [ ]:
# infl

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

In [ ]:
stmt = input()
probs = model.predict(prep_data(stmt))[0]

probs

dd


array([3.5075027e-05, 2.1006996e-04, 9.5064344e-05, 1.6913256e-04,
       6.4423610e-04, 3.3572639e-04, 9.9851066e-01], dtype=float32)

In [ ]:
import numpy as np

np.argmax(probs)

6

In [ ]:
def sentiment_score(tweet):
    prob = model.predict(prep_data(tweet))[0]
    mona = np.argmax(prob)+1
    if prob[int(mona)-1] >= 0.94:
      con ='Confident'
    # elif prob[int(mona[0])-1] >= 0.5:
    #   con ='Moderate'
    else:
      con ='Not confident'
    # return mona,prob[0][int(mona[0])-1]
    return mona,prob[int(mona)-1],con

In [ ]:
data = ['i want to marry you']

In [ ]:
sentiment_score(data)

(7, 0.9991487, 'Confident')

In [ ]:
sentiment_score(tt['tweet'].iloc[1])

(1, 0.8953059, 'Not confident')

In [ ]:
tt['sentiment']  = tt['tweet'].apply(lambda x: sentiment_score(x[:500])[0])


In [ ]:
tt['probability'] = tt['tweet'].apply(lambda x: sentiment_score(x[:500])[1])

In [ ]:
tt['Condition'] = tt['tweet'].apply(lambda x: sentiment_score(x[:500])[2])

In [ ]:
tt

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,sentiment,probability,Condition
0,1418257821041414145,1418257821041414145,1.626974e+12,2021-07-22 17:13:09,+0000,,My beautiful queen ❤️ https://t.co/Jmbld6ySq7,en,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,4,17,https://twitter.com/Cristiano/status/141825782...,[],[https://pbs.twimg.com/media/E66p-BrXIAMhsTS.jpg],1,https://pbs.twimg.com/media/E66p-BrXIAMhsTS.jpg,False,450689,6422,18890,,None,,,,,,,[],,,,,5,0.787187,Not confident
1,1416076947680026633,1416076947680026633,1.626454e+12,2021-07-16 16:47:08,+0000,,A good weekend to everyone 😎🙏🏽 https://t.co/B...,en,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,5,16,https://twitter.com/Cristiano/status/141607694...,[],[https://pbs.twimg.com/media/E6bqeiUXoAk6z5L.jpg],1,https://pbs.twimg.com/media/E6bqeiUXoAk6z5L.jpg,False,469456,8034,19990,,None,,,,,,,[],,,,,1,0.895306,Not confident
2,1414933377434767361,1414933377434767361,1.626181e+12,2021-07-13 13:03:00,+0000,,My love 💙 https://t.co/PH7gPrWYxT,en,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,2,13,https://twitter.com/Cristiano/status/141493337...,[],[https://pbs.twimg.com/media/E6LaZ7PXsAIss3M.jpg],1,https://pbs.twimg.com/media/E6LaZ7PXsAIss3M.jpg,False,537655,6528,21653,,None,,,,,,,[],,,,,5,0.609789,Not confident
3,1414194896219648000,1414194896219648000,1.626005e+12,2021-07-11 12:08:32,+0000,,"Se você tiver o “ter” sem o “ser”, não pertenc...",pt,[lr],[],155659213,155659213,Cristiano,Cristiano Ronaldo,7,12,https://twitter.com/Cristiano/status/141419489...,[],[https://pbs.twimg.com/media/E6A6wskXIAAo7kf.jpg],1,https://pbs.twimg.com/media/E6A6wskXIAAo7kf.jpg,False,186094,5033,9625,,None,,,,,,,[],,,,,5,0.565301,Not confident
4,1413187283512176642,1413187283512176642,1.625765e+12,2021-07-08 17:24:38,+0000,,Peace of mind 🌊 https://t.co/LuCcZ8DlaC,en,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,4,17,https://twitter.com/Cristiano/status/141318728...,[],[https://pbs.twimg.com/media/E5ymVsMX0AEnG8j.jpg],1,https://pbs.twimg.com/media/E5ymVsMX0AEnG8j.jpg,False,264358,3560,11422,,None,,,,,,,[],,,,,5,0.745395,Not confident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1326853175274561537,1326853175274561537,1.605181e+12,2020-11-12 11:43:43,+0000,,Gracias por llegar a nuestras vidas y llenarlo...,es,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,4,11,https://twitter.com/Cristiano/status/132685317...,[],[https://pbs.twimg.com/media/Emnt7S-XIAAFDo_.jpg],1,https://pbs.twimg.com/media/Emnt7S-XIAAFDo_.jpg,False,233988,2693,5824,,None,,,,,,,[],,,,,5,0.738190,Not confident
96,1326664085308575744,1326664085308575744,1.605136e+12,2020-11-11 23:12:20,+0000,,Feliz por atingir mais uma marca histórica com...,pt,[],[],155659213,155659213,Cristiano,Cristiano Ronaldo,3,23,https://twitter.com/Cristiano/status/132666408...,[],[https://pbs.twimg.com/media/EmlB83rXMAQJGpR.j...,1,https://pbs.twimg.com/media/EmlB83rXMAQJGpR.jpg,False,190823,1944,9687,,None,,,,,,,[],,,,,5,0.413295,Not confident
97,1326170869840949248,1326170869840949248,1.605019e+12,2020-11-10 14:32:29,+0000,,Feliz por estar de volta a casa! 🇵🇹💪🏽 #todospo...,pt,[todosportugal],[],155659213,155659213,Cristiano,Cristiano Ronaldo,2,14,https://twitter.com/Cristiano/status/132617086...,[],[https://pbs.twimg.com/media/EmeBX1wXYAA0QRV.j...,1,https://pbs.twimg.com/media/EmeBX1wXYAA0QRV.jpg,False,147625,1787,5892,,None,,,,,,,[],,,,,5,0.623682,Not confident
98,1325467261239431170,1325467261239431170,1.604851e+12,2020-11-08 15:56:35,+0000,,Head up and believe in the work we’re doing! 💪...,en,[finoallafine],[],155659213,155659213,Cristiano,C

In [ ]:
import pandas as pd

In [ ]:
tt.to_excel("cial6_tinder_tweet_10000.xlsx")

In [ ]:
# tt.iloc[:, [7,39,40,41]].values

In [ ]:
dataset = pd.read_excel('/content/cial6_tinder_tweet_10000.xlsx')
X_train = dataset.iloc[:, [7,39,40,41]].values

In [ ]:
title = ['Tweet', 'Sentiment', 'Probability', 'Confidence']

df = pd.DataFrame(X_train, columns = title)
df

,Tweet,Sentiment,Probability,Confidence
0,My beautiful queen ❤️ https://t.co/Jmbld6ySq7,5,0.787187,Not confident
1,A good weekend to everyone 😎🙏🏽 https://t.co/B...,1,0.895306,Not confident
2,My love 💙 https://t.co/PH7gPrWYxT,5,0.609789,Not confident
3,"Se você tiver o “ter” sem o “ser”, não pertenc...",5,0.565301,Not confident
4,Peace of mind 🌊 https://t.co/LuCcZ8DlaC,5,0.745395,Not confident
...,...,...,...,...
95,Gracias por llegar a nuestras vidas y llenarlo...,5,0.73819,Not confident
96,Feliz por atingir mais uma marca histórica com...,5,0.413295,Not confident
97,Feliz por estar de volta a casa! 🇵🇹💪🏽 #todospo...,5,0.623682,Not confident
98,Head up and believe in the work we’re doing! 💪...,5,0.532002,Not confident


In [ ]:
# pd.DataFrame(X_train).to_excel("tesla_tweets.xlsx", header=False, index=False)

df.to_excel("/content/cial6_tinder_tweet_10000.xlsx")

In [ ]:
# df = pd.DataFrame(dataset, columns= ['Tweet','Sentiment','Prabability', 'Confidence'])
confident_tesla = df.loc[df['Confidence'] == 'Confident']
print (confident_tesla)

                                                Tweet  ... Confidence
28  The usual 🤷🏽‍♂️😅 ssssiiiimmmm 💪🏽 #winningteam ...  ...  Confident
73  One last Christmas gift: Today we reopen @Pest...  ...  Confident
74  We wish you a Merry Christmas!! Full of love, ...  ...  Confident

[3 rows x 4 columns]


In [ ]:
confident_tesla.to_excel("/content/test_data(Marriott Hotels).xlsx")